In [1]:
# !git clone https://github.com/wasef-c/EmoSpeech.git
# !pip install transformers[torch] torch datasets seaborn matplotlib scikit-learn
# !pip install --upgrade Pillow
# TRY WEIGHING IMAGE MDEL HIGEHR
import os
import logging
import warnings
import random

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from transformers import (
    AutoImageProcessor,
    ViTForImageClassification,
    BertModel,
    AutoTokenizer,
    get_scheduler,
    DinatForImageClassification,
)
from datasets import load_dataset
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts

import os
# Custom imports from your "functions_older.py" (adjust as needed)
from functions_old import *
def compute_regression_metrics(predictions, labels):
    """
    For regression: compute CCC and R^2.

    eval_pred: (predictions, labels)
       predictions: np.array of shape (N,) or (N,1)
       labels:      np.array of shape (N,) or (N,1)
    """
    # predictions, labels = eval_pred

    # Make sure arrays are 1D if your model outputs (N,1)
    predictions = np.squeeze(predictions)
    labels = np.squeeze(labels)

    ccc = concordance_correlation_coefficient(predictions, labels)
    r2 = r2_score(labels, predictions)

    return {
        "CCC": ccc,
        "R^2": r2
    }
# Suppress warnings
warnings.filterwarnings("ignore", category=UserWarning)

logging.getLogger().addHandler(logging.NullHandler())
logging.getLogger("natten.functional").setLevel(logging.ERROR)

def collate_fn_reg(examples, column):
    """
    Custom collate function to handle batching of image data and BERT inputs.
    """
    pixel_values = torch.stack([example["pixel_values"] for example in examples]).to(device)
    input_ids = torch.stack([example["input_ids"] for example in examples]).to(device)
    attention_mask = torch.stack([example["attention_mask"] for example in examples]).to(device)
    labels = torch.tensor([example[column] for example in examples]).to(device)
    bert_embeddings = torch.stack([example["bert_embeddings"] for example in examples]).to(device)



    return {
        "pixel_values": pixel_values,
        "input_ids": input_ids,
        "attention_mask": attention_mask,
        "labels": labels,
        "bert_embeddings":bert_embeddings
    }
    
warnings.filterwarnings("ignore", category=UserWarning)
logging.getLogger().addHandler(logging.NullHandler())

# ----------------------------------------------------------------------
# Configuration and Paths
# ----------------------------------------------------------------------
# r"C:\Users\Paolo\Documents\carol_emo_rec\MLLM\Currciulum_Models\Speaker\20250107_6\best_model.pt"
column = "EmoDom" #"EmoVal" ## or "arousal", "score", etc."EmoVal" # 

# checkpoint_path = "./DinatCurriculum/Regression/Activation/EmoAct/20250117_1/best_model.pt"  #"./DinatCurriculum/Regression/Valence/20250114_10/best_model.pt"# None #"./Curriculum/Regression/Valence/20250110_3/best_model.pt" #"./Curriculum/Regression/Activation/20250109_6/best_model.pt"
checkpoint_path = "/home/rml/Documents/pythontest/Trained_Models/curr_V2/EmoAct/20250307_1/best_model.pt" #None #"/home/rml/Documents/pythontest/Trained_Models/curr/EmoAct/20250305_1/best_model.pt"
base_dir = os.path.join("/home/rml/Documents/pythontest/Trained_Models/curr_V2", column)
output_dir = create_unique_output_dir(base_dir)
os.makedirs(output_dir, exist_ok=True)



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

BATCH_SIZE = 64
LEARNING_RATE = 1e-5
EPOCHS = 20
WEIGHT_DECAY = 0.01
PATIENCE = 5

# Pre-trained model names (ViT + BERT)
image_model_name = "shi-labs/dinat-mini-in1k-224"
bert_model_name = "bert-base-uncased"

# ----------------------------------------------------------------------
# Load Dataset
# ----------------------------------------------------------------------
dataset_name = 'cairocode/MSPP_MEL_6'
dataset_name = 'cairocode/MSPP_SPLIT_MEL2'
dataset = load_dataset(dataset_name)


# Define a filtering function
def is_valid_transcript(example):
    return isinstance(example['transcript'], str)

# Filter out non-string transcripts in each split
filtered_dataset = dataset.filter(is_valid_transcript)

# seed = 42
# random.seed(seed)
# np.random.seed(seed)
# unique_speakers = set(filtered_dataset['train']['SpkrID'])
# shuffled_speakers = list(unique_speakers)
# random.shuffle(shuffled_speakers)

# # Split speakers into 80% (train+val) and 20% (test)
# split_idx = int(len(shuffled_speakers) * 0.8)
# train_val_speakers = shuffled_speakers[:split_idx]
# test_speakers = shuffled_speakers[split_idx:]

# # Split the train_val set into 85% train and 15% validation
# train_val_split_idx = int(len(train_val_speakers) * 0.85)
# train_speakers = train_val_speakers[:train_val_split_idx]
# val_speakers = train_val_speakers[train_val_split_idx:]

# # Filter the dataset based on speaker IDs
# train_dataset = filtered_dataset['train'].filter(lambda example: example['SpkrID'] in train_speakers)
# val_dataset = filtered_dataset['train'].filter(lambda example: example['SpkrID'] in val_speakers)
# test_dataset = filtered_dataset['train'].filter(lambda example: example['SpkrID'] in test_speakers)
train_dataset = filtered_dataset['train']
val_dataset = filtered_dataset['validation']
test_dataset = filtered_dataset['test']


# def rename_to_label(example):
# #     # Convert to float if necessary
#      return {"r": float(example[column])}


# # 2) Map the transformation
# dataset = dataset.map(rename_to_label)
# train_dataset = dataset['train']
# val_dataset = dataset['validation']
# test_dataset = dataset['test']

# ----------------------------------------------------------------------
# Set Transforms and Prepare Dataloaders
# ----------------------------------------------------------------------
train_dataset.set_transform(train_transforms)
val_dataset.set_transform(val_transforms)
test_dataset.set_transform(val_transforms)

train_loader = DataLoader(
    train_dataset,
    # sampler=CustomSampler(train_dataset),
    batch_size=BATCH_SIZE,
    collate_fn=lambda examples: collate_fn_reg(examples, column=column),
)
val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    collate_fn=lambda examples: collate_fn_reg(examples, column=column),
)
test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    collate_fn=lambda examples: collate_fn_reg(examples, column=column),
)

# ----------------------------------------------------------------------
# Initialize Image + Text Models
# ----------------------------------------------------------------------
    
image_model = DinatForImageClassification.from_pretrained(
    image_model_name,
    num_labels=1,
    ignore_mismatched_sizes=True,
    problem_type="regression",
).to(device)

image_processor = DinatForImageClassification.from_pretrained(image_model_name).to(device)


tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
base_bert_model = BertModel.from_pretrained(bert_model_name).to(device)

# ----------------------------------------------------------------------
# Initialize the Combined Regression Model
# ----------------------------------------------------------------------
model = CombinedModelsBi(
    image_model=image_model,
    bert_model=bert_model,
    image_feature_dim=512,
    bert_embedding_dim=768,
    combined_dim=512,
    num_labels=1,

).to(device)


if checkpoint_path != None:
    checkpoint = torch.load(checkpoint_path, map_location=device)

    # # Option 1: Print each key on a separate line
    # # for key in checkpoint.keys():
    # #     print(key)

    # Define the keywords to include and exclude
    include_keyword = "image_model"
    exclude_keys = {
        "image_model.classifier.weight",
        "image_model.classifier.bias"
    }

    # Use dictionary comprehension to filter the keys
    filtered_checkpoint = {
        key: value for key, value in checkpoint.items()
        if include_keyword in key and key not in exclude_keys
    }

    # # Optional: Verify the filtered keys
    # print("Filtered keys to be loaded:")
    # for key in filtered_checkpoint.keys():
    #     print(f"- {key}")

    # Load the filtered state dict
    model.load_state_dict(filtered_checkpoint, strict=False)


# ----------------------------------------------------------------------
# Training Setup
# ----------------------------------------------------------------------
optimizer = optim.AdamW(
    model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
num_training_steps = len(train_loader) * EPOCHS
# lr_scheduler = get_scheduler(
#     "linear",
#     optimizer=optimizer,
#     num_warmup_steps=0,
#     num_training_steps=num_training_steps,
# )
lr_scheduler = CosineAnnealingWarmRestarts(
    optimizer,
    T_0=10,  # Number of iterations for the first restart
    T_mult=2,  # Multiplicative factor for subsequent restart periods
)

# import torch
# import torch.nn as nn


class CCCLoss(nn.Module):
    def __init__(self):
        super(CCCLoss, self).__init__()

    def forward(self, y_pred, y_true):
        y_true_mean = torch.mean(y_true)
        y_pred_mean = torch.mean(y_pred)

        y_true_var = torch.var(y_true, unbiased=False)
        y_pred_var = torch.var(y_pred, unbiased=False)

        covariance = torch.mean((y_true - y_true_mean)
                                * (y_pred - y_pred_mean))

        ccc = (2 * covariance) / (
            y_true_var + y_pred_var + (y_true_mean - y_pred_mean) ** 2 + 1e-8
        )  # Adding epsilon to avoid division by zero

        loss = 1 - ccc
        return loss


criterion = CCCLoss()

patience_counter = 0
best_val_loss = 0 # float("inf")

train_losses = []
val_losses = []
epochs_list = []
best_model_path = os.path.join(output_dir, "best_model.pt")

# ----------------------------------------------------------------------
# Training Loop
# ----------------------------------------------------------------------
for epoch in range(EPOCHS):
    model.train()
    train_loss = 0.0
    progress_bar = tqdm(
        train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}", leave=False)

    for batch in progress_bar:
        pixel_values = batch["pixel_values"].to(device)
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].float().to(device)  # Ensure labels are float for regression
    
        # Forward pass
        outputs_dict = model(
            pixel_values=pixel_values,
            bert_input_ids=input_ids,
            bert_attention_mask=attention_mask,
            labels=labels  # Provide labels for DCCA loss
        )
    
        # Predictions and primary regression loss
        predictions = outputs_dict["logits"].squeeze(-1)
        regression_loss = criterion(predictions, labels)
        # # Weighted total loss
        # lambda_dcca = min(0.0, epoch / 50)  # Gradually increase
        # lambda_recon = 0.0 #min(0.1, epoch / 500)

        # total_loss = (
        #     regression_loss
        #     + lambda_dcca * dcca_loss
        #     + lambda_recon * reconstruction_loss
        # )
        total_loss = regression_loss
        # Backpropagation
        optimizer.zero_grad()
        total_loss.backward()
        optimizer.step()
    
        # Accumulate and display losses in progress bar
        train_loss += total_loss.item()
        progress_bar.set_postfix({
            "total_loss": total_loss.item(),
        })
    
    
    avg_train_loss = train_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{EPOCHS} - Training Loss: {avg_train_loss:.4f}")
    lr_scheduler.step()

    # ------------------------------------------------------------------
    # Validation
    # ------------------------------------------------------------------
    model.eval()
    val_loss = 0.0
    all_predictions = []
    all_labels = []

    with torch.no_grad():
        for batch in val_loader:
            pixel_values = batch["pixel_values"].to(device)
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].float().to(device)

            outputs_dict = model(
                pixel_values=pixel_values,
                bert_input_ids=input_ids,
                bert_attention_mask=attention_mask
            )
            predictions = outputs_dict["logits"].squeeze(-1)
            loss = criterion(predictions, labels)
            val_loss += loss.item()

            all_predictions.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    avg_val_loss = val_loss / len(val_loader)
    train_losses.append(avg_train_loss)
    val_losses.append(avg_val_loss)
    epochs_list.append(epoch + 1)
    metrics = compute_regression_metrics(all_predictions, all_labels)

    print(f"Validation Loss: {avg_val_loss:.4f} Metrics: {metrics}")

    # Optional: compute regression metrics if you have them
    # metrics_dict = compute_regression_metrics(all_predictions, all_labels)
    # print(metrics_dict)

    # Early stopping based on validation loss
    # if avg_val_loss < best_val_loss:
    #     best_val_loss = avg_val_loss
    #     patience_counter = 0
    #     torch.save(model.state_dict(), best_model_path)
    #     print("Validation loss improved. Saving best model and resetting patience counter.")
    # else:
    #     patience_counter += 1
    #     if patience_counter >= PATIENCE:
    #         print("Early stopping triggered. Stopping training.")
    #         break
    if metrics['CCC'] > best_val_loss:
        best_val_loss = metrics['CCC']
        patience_counter = 0
        torch.save(model.state_dict(), best_model_path)
        print("Validation loss improved. Saving best model and resetting patience counter.")
    else:
        patience_counter += 1
        if patience_counter >= PATIENCE:
            print("Early stopping triggered. Stopping training.")
            break

# ----------------------------------------------------------------------
# Load Best Model for Final Evaluation
# ----------------------------------------------------------------------
print("Loading best model for final evaluation.")
model.load_state_dict(torch.load(best_model_path))
model.to(device)

# ----------------------------------------------------------------------
# Test Evaluation
# ----------------------------------------------------------------------
print("\nStarting Test Evaluation...")
model.eval()
test_loss = 0.0
all_test_predictions = []
all_test_labels = []

with torch.no_grad():
    test_progress_bar = tqdm(test_loader, desc="Testing", leave=False)
    for batch in test_progress_bar:
        pixel_values = batch["pixel_values"].to(device)
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].float().to(device)

        outputs_dict = model(
            pixel_values=pixel_values,
            bert_input_ids=input_ids,
            bert_attention_mask=attention_mask
        )
        predictions = outputs_dict["logits"].squeeze(-1)

        loss = criterion(predictions, labels)
        test_loss += loss.item()

        all_test_predictions.extend(predictions.cpu().numpy())
        all_test_labels.extend(labels.cpu().numpy())

avg_test_loss = test_loss / len(test_loader)
metrics = compute_regression_metrics(all_test_predictions, all_test_labels)

print(f"Test Loss: {avg_test_loss:.4f} Metrics: {metrics}")

# If you have your own regression metrics function:
# test_metrics = compute_regression_metrics(all_test_predictions, all_test_labels)
# print("Test Metrics:", test_metrics)

# ----------------------------------------------------------------------
# Save Final Results & Metadata
# ----------------------------------------------------------------------
final_metrics_str = f"Test Loss: {avg_test_loss:.4f} Metrics: {metrics}"
save_training_metadata(
    output_dir,
    Pretrain_file = output_dir,
    dataset_name=dataset_name,
    model_type = "DINAT DCCA",
    results = final_metrics_str,
    column = column,
    speaker_disentanglement = True,
    metrics = final_metrics_str,
    weight_decay = WEIGHT_DECAY,
    class_weights=None,
    speakers = None,
)

metrics_file = os.path.join(output_dir, "metrics.txt")
with open(metrics_file, "w") as f:
    f.write(final_metrics_str + "\n")

print(f"Metrics saved to {metrics_file}")


/home/rml/Documents/pythontest/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Some weights of DinatForImageClassification were not initialized from the model checkpoint at shi-labs/dinat-mini-in1k-224 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 512]) in the checkpoint and torch.Size([1, 512]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([1]) in the m

Epoch 1/20 - Training Loss: 0.5734
Validation Loss: 0.5172 Metrics: {'CCC': np.float32(0.6010388), 'R^2': 0.18839091062545776}
Validation loss improved. Saving best model and resetting patience counter.


Epoch 2/20 - Training Loss: 0.5242
Validation Loss: 0.5124 Metrics: {'CCC': np.float32(0.6013467), 'R^2': 0.22934812307357788}
Validation loss improved. Saving best model and resetting patience counter.


Epoch 3/20 - Training Loss: 0.5013
Validation Loss: 0.5086 Metrics: {'CCC': np.float32(0.6040397), 'R^2': 0.24072754383087158}
Validation loss improved. Saving best model and resetting patience counter.


Epoch 4/20 - Training Loss: 0.4806
Validation Loss: 0.5106 Metrics: {'CCC': np.float32(0.6004127), 'R^2': 0.2573062777519226}


...

Epoch 7/20 - Training Loss: 0.4246
Validation Loss: 0.5311 Metrics: {'CCC': np.float32(0.566581), 'R^2': 0.31355226039886475}


Epoch 8/20 - Training Loss: 0.4120
Validation Loss: 0.5365 Metrics: {'CCC': np.float32(0.5544069), 'R^2': 0.32086455821990967}
Early stopping triggered. Stopping training.
Loading best model for final evaluation.


/tmp/ipykernel_780874/3164830471.py:415: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(best_model_path))



Starting Test Evaluation...


Test Loss: 0.5800 Metrics: {'CCC': np.float32(0.562535), 'R^2': 0.19434547424316406}
Training metadata saved successfully at: /home/rml/Documents/pythontest/Trained_Models/curr_V2/EmoDom/20250310_1/training_metadata.txt
Metrics saved to /home/rml/Documents/pythontest/Trained_Models/curr_V2/EmoDom/20250310_1/metrics.txt


In [2]:
from datasets import load_dataset
dataset_name = 'cairocode/MSPP_SPLIT_MEL2'
dataset = load_dataset(dataset_name)

In [3]:
import os

metrics_file = os.path.join(output_dir, "metrics.txt")
with open(metrics_file, "w") as f:
    f.write(final_metrics_str + "\n")

print(f"Metrics saved to {metrics_file}")

Metrics saved to /home/rml/Documents/pythontest/Trained_Models/curr_V2/EmoDom/20250310_1/metrics.txt


In [4]:
# from datasets import DatasetDict
# from sklearn.model_selection import train_test_split
# import csv
# from functions_older import *
# import datasets
# from datasets import load_dataset
# import os
# import logging
# import warnings
# import random

# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import DataLoader
# from transformers import (
#     AutoImageProcessor,
#     ViTForImageClassification,
#     BertModel,
#     AutoTokenizer,
#     get_scheduler,
# )
# from datasets import load_dataset
# import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# from tqdm.auto import tqdm

# def collate_fn_test(examples):
#     """
#     Custom collate function to handle batching of image data and BERT inputs.
#     """
#     # print(examples)
#     pixel_values = torch.stack([example["pixel_values"]
#                                for example in examples]).to(device)
#     input_ids = torch.stack([example["input_ids"]
#                             for example in examples]).to(device)
#     attention_mask = torch.stack(
#         [example["attention_mask"] for example in examples]).to(device)
#     files = [example["file"] for example in examples]

#     bert_embeddings = torch.stack(
#         [example["bert_embeddings"] for example in examples]).to(device)

#     return {
#         "pixel_values": pixel_values,
#         "input_ids": input_ids,
#         "attention_mask": attention_mask,
#         "files": files,
#         "bert_embeddings": bert_embeddings
#     }

# checkpoint_path = "./DinatCurriculum/Regression/Valence/20250116_19/best_model.pt"
# output_dir = "./DinatCurriculum/Regression/Valence/20250116_19/
# os.makedirs(output_dir, exist_ok=True)


# dataset_name = 'cairocode/MSPP_TEST_SYL'
# # test_dataset = 
# test_dataset = load_dataset(dataset_name)['test']

# test_dataset.set_transform(test_transforms)

# test_loader = DataLoader(
#     test_dataset,
#     batch_size=50,
#     collate_fn=collate_fn_test,
# )



# image_model_name = "google/vit-base-patch16-224"
# bert_model_name = "bert-base-uncased"

# image_processor = AutoImageProcessor.from_pretrained(image_model_name)
# image_model = ViTForImageClassification.from_pretrained(
#     image_model_name,
#     ignore_mismatched_sizes=True,
#     num_labels=1,
#     problem_type="regression"
# ).to(device)

# tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
# base_bert_model = BertModel.from_pretrained(bert_model_name).to(device)

# # ----------------------------------------------------------------------
# # Initialize the Combined Regression Model
# # ----------------------------------------------------------------------
# model = CombinedModelsBi(
#     image_model=image_model,
#     bert_model=bert_model,
#     image_feature_dim=768,   # Feature dim of ViT
#     bert_embedding_dim=768,  # BERT embedding dim
#     combined_dim=1024,       # Combined dimension
#     num_labels=1
# ).to(device)


# if checkpoint_path != None:
#     checkpoint = torch.load(checkpoint_path, map_location=device)


# model.load_state_dict(torch.load(checkpoint_path))
# model.to(device)




# # ----------------------------------------------------------------------
# # Test Evaluation
# # ----------------------------------------------------------------------
# print("\nStarting Test Evaluation...")
# model.eval()
# test_loss = 0.0
# all_test_predictions = []
# all_test_labels = []
# all_filenames = []  # To store filenames

# with torch.no_grad():
#     test_progress_bar = tqdm(test_loader, desc="Testing", leave=False)
#     for batch in test_progress_bar:
#         pixel_values = batch["pixel_values"].to(device)
#         input_ids = batch["input_ids"].to(device)
#         attention_mask = batch["attention_mask"].to(device)
#         # labels = batch["labels"].float().to(device)

#         # Extract filenames from the batch (assuming "filename" is part of batch)
#         filenames = batch["files"]  # Adjust key as necessary
#         all_filenames.extend(filenames)

#         outputs_dict = model(
#             pixel_values=pixel_values,
#             bert_input_ids=input_ids,
#             bert_attention_mask=attention_mask
#         )
#         predictions = outputs_dict["logits"].squeeze(-1)

#         # loss = criterion(predictions, labels)
#         # test_loss += loss.item()

#         all_test_predictions.extend(predictions.cpu().numpy())
#         # all_test_labels.extend(labels.cpu().numpy())



# output_csv_path = os.path.join(output_dir, "test_predictions.csv")
# with open(output_csv_path, mode="w", newline="") as csv_file:
#     writer = csv.writer(csv_file)
#     writer.writerow(["Filename", "Domination"])
#     for filename, prediction in zip(all_filenames, all_test_predictions):
#         writer.writerow([filename, prediction])

# print(f"Predictions saved to {output_csv_path}")

# # print(dataset_dict)


In [5]:
# import pandas as pd

# # Step 1: Define a function to process a single CSV
# # Step 1: Define a function to process a single CSV
# def process_csv(file_path):
#     # Load the CSV into a DataFrame
#     df = pd.read_csv(file_path)

#     # Group by 'Filename' and calculate the mean for numeric columns
#     df_mean = df.groupby('FileName', as_index=False).mean()

#     # Clip values to the range [1, 7]
#     for column in df_mean.columns:
#         if df_mean[column].dtype in ['float64', 'int64']:
#             df_mean[column] = df_mean[column].clip(lower=1, upper=7)

#     return df_mean

# # Step 2: Process multiple CSVs and merge them
# def merge_csvs(file_paths, output_path):
#     processed_dfs = []

#     # Process each file
#     for file_path in file_paths:
#         processed_dfs.append(process_csv(file_path))

#     # Merge all processed DataFrames on 'Filename'
#     merged_df = processed_dfs[0]
#     for df in processed_dfs[1:]:
#         merged_df = pd.merge(merged_df, df, on='FileName', how='inner')

#     # Save the merged DataFrame to a new CSV
#     merged_df.to_csv(output_path, index=False)
#     return merged_df

# # Step 3: Specify the input file paths and output file path
# file_paths = [
#     r"D:\Downloads\MelRegression\MelRegression\EmoAct\20250122_1\test_predictions.csv",
#     r"D:\Downloads\MelRegression\MelRegression\EmoVal\20250122_2\test_predictions.csv",
#     r"D:\Downloads\MelRegression\MelRegression\EmoDom\20250123_1\test_predictions.csv",
# ]
# output_path = r"D:\Downloads\MelRegression\output.csv"

# # Merge the CSVs
# df = merge_csvs(file_paths, output_path)

# print(f"Merged CSV saved to {output_path}")


In [6]:
# import pandas as pd

# # # Read the CSV file
# # df = pd.read_csv("your_file.csv")

# # Change column names
# # Option 1: Rename specific columns
# df.rename(columns={"Filename": "FileName",
#                    "Activation": "EmoAct", 
#                    "Valence": "EmoVal",
#                    "Domination": "EmoDom",
#                   }, inplace=True)

# # Save the updated DataFrame back to CSV
# df.to_csv("./updated_file.csv", index=False)

# print("Column names updated successfully!")
